In [1]:
%load_ext autoreload

In [2]:
%autoreload

import h2o
import pandas as pd
import seaborn as sns
import utilities as utils

from h2o.automl import H2OAutoML

### Import data and format with utilities.py

In [3]:
data_list = utils.read_data()
Data = utils.DataProcessor(data_list)
df = Data.create_train_data()
df.sort_index(inplace=True)
df = df.loc['2019-09-04 08:00:00':]

df.head()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_w_1000hPa:ms,hour,day_of_week,quarter,month,year,day_of_year,day_of_month,building,pv_measurement
date_forecast,,,,,,,,,,,,,,,,,,,,,
2019-09-04 08:00:00,6.9,1.231,2103.100098,1166053.375,382.100006,2103.100098,0.0,278.600006,66.400002,226964.906250,...,-0.0,8,2,3,9,2019,247,4,1,570.1125
2019-09-04 08:00:00,6.8,1.226,2104.000000,1166048.250,382.100006,2104.000000,0.0,278.600006,66.400002,226962.703125,...,-0.0,8,2,3,9,2019,247,4,0,3786.2000
2019-09-04 08:00:00,6.7,1.223,2122.699951,1167217.875,382.500000,2122.699951,0.0,278.399994,66.099998,225542.296875,...,-0.0,8,2,3,9,2019,247,4,2,137.2000
2019-09-04 09:00:00,6.7,1.225,2476.500000,1559157.375,479.700012,2476.500000,0.0,278.399994,73.199997,251260.000000,...,-0.0,9,2,3,9,2019,247,4,1,542.5125
2019-09-04 09:00:00,6.7,1.220,2478.399902,1559161.250,479.700012,2478.399902,0.0,278.399994,73.199997,251256.593750,...,-0.0,9,2,3,9,2019,247,4,0,3400.7600


In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 21+35-LTS-2513, mixed mode, sharing)
  Starting server from C:\Users\elias\AppData\Local\Programs\Python\Python310\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\elias\AppData\Local\Temp\tmpfjf_2nsg
  JVM stdout: C:\Users\elias\AppData\Local\Temp\tmpfjf_2nsg\h2o_elias_started_from_python.out
  JVM stderr: C:\Users\elias\AppData\Local\Temp\tmpfjf_2nsg\h2o_elias_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.4
H2O_cluster_version_age:,8 days
H2O_cluster_name:,H2O_from_python_elias_b1253j
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.980 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


## Feature Engineering with Featurewiz
Featurewiz not working on Windows desktop. Use features from featurewiz from Ubuntu.

In [5]:
FEATURES = ['elevation:m', 'sun_elevation:d', 'direct_rad_1h:J', 'sun_azimuth:d', 'effective_cloud_cover:p', 'msl_pressure:hPa', 't_1000hPa:K', 'day_of_year', 'wind_speed_u_10m:ms', 'ceiling_height_agl:m',
            'visibility:m', 'relative_humidity_1000hPa:p', 'year', 'day_of_month', 'day_of_week', 'fresh_snow_24h:cm', 'fresh_snow_6h:cm', 'fresh_snow_1h:cm', 'pv_measurement']

X_train_wiz = df[FEATURES]

## AutoML with H2O

In [6]:
data = X_train_wiz.copy()

# Convert to h2o frame
train = h2o.H2OFrame(data)
y = 'pv_measurement'
X = train.columns
X.remove(y)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
# Initialize and train the H2OAutoML model
aml = H2OAutoML(max_models=30, seed=1)
aml.train(x=X, y=y, training_frame=train)

AutoML progress: |
08:55:55.949: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),5/30
# GBM base models (used / total),5/14
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),0/13
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [8]:
# View the AutoML Leaderboard
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))

model_id                                                   rmse       mse       mae       rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20231012_85555     224.973   50613.1   76.3678  nan                          50613.1
StackedEnsemble_BestOfFamily_1_AutoML_1_20231012_85555  229.328   52591.1   73.8032  nan                          52591.1
GBM_grid_1_AutoML_1_20231012_85555_model_5              229.629   52729.4   73.1464  nan                          52729.4
GBM_4_AutoML_1_20231012_85555                           231.437   53563.2   79.926   nan                          53563.2
GBM_3_AutoML_1_20231012_85555                           236.795   56071.7   82.0656  nan                          56071.7
GBM_1_AutoML_1_20231012_85555                           238.586   56923.2   85.4493  nan                          56923.2
GBM_2_AutoML_1_20231012_85555                           243.95    59511.6   85.3648  nan                          59511.6
DRF_1_AutoML_1_20231012_

In [18]:
# Predict on the test data
test_pd = Data.create_test_data()
test = h2o.H2OFrame(test_pd)

preds = aml.predict(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [19]:
# Cast preds to pd.DataFrame and save to csv
preds_df = preds.as_data_frame()
# Set all negative values to 0
preds_df[preds_df < 0] = 0
# Data formatting for Kaggle delivery
results = pd.concat([test_pd.reset_index(), preds_df], axis=1)
results['date_forecast'] = pd.to_datetime(results['date_forecast'])
results.set_index('date_forecast', inplace=True)
# Rename column 'predict' to 'pv_prediction' in results
results.rename(columns={'predict': 'pv_prediction'}, inplace=True)

utils.save_to_csv(results, 'stacked_ensemble_h2o')